### Extract Features from the Network Traffic

Load the `netflix.pcap` file, which is a packet trace that includes network traffic. 

Click [here](https://github.com/noise-lab/ml-systems/blob/main/docs/notebooks/data/netflix.pcap) to download `netflix.pcap`.


In [ ]:
from scapy.all import rdpcap, IP, TCP, UDP, DNS
import pandas as pd

# Read pcap and build rows with a robust 'length' computation
pkts = rdpcap("../../myData/netflix.pcap")
rows = []
for pkt in pkts:
    is_DNS = DNS in pkt
    proto = "DNS" if is_DNS else ("TCP" if TCP in pkt else ("UDP" if UDP in pkt else "Other"))
    # compute packet length robustly; len(pkt) normally works for scapy packets
    try:
        pkt_len = len(pkt)
    except Exception:
        pkt_len = getattr(pkt, 'len', None)
    rows.append({
        "timestamp": float(getattr(pkt, "time", 0)),
        "length": pkt_len,
        "src_ip": pkt[IP].src if IP in pkt else None,
        "dst_ip": pkt[IP].dst if IP in pkt else None,
        "txn_id": pkt[DNS].id if is_DNS else None,
        "protocol": proto,
        "src_port": pkt.sport if (TCP in pkt or UDP in pkt) else None,
        "dst_port": pkt.dport if (TCP in pkt or UDP in pkt) else None,
        "info": str(pkt.summary())
    })
pcap__import = pd.DataFrame(rows)

# Normalise: ensure there is a numeric 'length' column for downstream processing
if 'length' not in pcap__import.columns and 'len' in pcap__import.columns:
    pcap__import['length'] = pcap__import['len']
# coerce to numeric and use pandas nullable integer dtype where possible
if 'length' in pcap__import.columns:
    pcap__import['length'] = pd.to_numeric(pcap__import['length'], errors='coerce').astype('Int64')



In [ ]:
netflix_dns = pcap__import[
    pcap__import["info"].str.contains(r"netflix|nflx", case=False, na=False)
]
print(netflix_dns.head(15))

### Identifying the Service Type

Use the DNS traffic to filter the packet trace for Netflix traffic.

In [ ]:
# only netflix queries (not answers)
netflix_qry = pcap__import[
    pcap__import["info"].str.contains(r"(netflix|nflx)", case=False, na=False)
    & pcap__import["info"].str.contains("Qry", case=False, na=False)
]

# show the txn ids and a few context columns
print(netflix_qry[["timestamp","src_ip","dst_ip","txn_id","info"]].head(30))
#drop duplicates
netflix_qry = netflix_qry.drop_duplicates(subset=["txn_id"])
print(netflix_qry[["timestamp","src_ip","dst_ip","txn_id","info"]].head(30))
#create a set of txn ids
netflix_txn_ids = set(netflix_qry["txn_id"])
print(netflix_txn_ids)


In [ ]:
#now we want to loop through netflix_dns and find all the answers that match the txn ids
nflx_answers = pcap__import[pcap__import["txn_id"].isin(netflix_txn_ids) & pcap__import["info"].str.contains("Ans", case=False, na=False) & pcap__import["protocol"].eq("DNS")] 
nflx_answers.head()
print(len(nflx_answers))

In [ ]:
#now let us extract the ips of nflx answers
nflx_ips = set()
for info in nflx_answers["info"]:
    parts = info.split("\"")
    for part in parts:
        if part.count(".") == 3:  # crude check for an IP address
            nflx_ips.add(part)  
print(nflx_ips)

### Generate Statistics

Generate statistics and features for the Netflix traffic flows. Use the `netml` library or any other technique that you choose to generate a set of features that you think would be good features for your model. 

In [ ]:
nflx_pkts = pcap__import[
    (pcap__import["src_ip"].isin(nflx_ips)) | (pcap__import["dst_ip"].isin(nflx_ips))
]

nflx_pkts.head()

#I want to get some statistics that I can use about the nflx packets and traffic flows
print(len(nflx_pkts))
print(nflx_pkts["length"].describe())

#now lets get some more statistics to help us truly understand the traffic flows and provide features for our MOdel
nflx_pkts.describe()

#I also want to see how to the packets are distributed over time
nflx_pkts["timestamp"].plot(kind='hist', bins=50, title='Distribution of Packet Timestamps', xlabel='Timestamp', ylabel='Frequency')
#other features that might be useful are inter-arrival times, packet sizes, burstiness, protocol distribution, flow durations, and packet counts per flow.
#inter arrival time: 
print("inter-arrival time statistics:")
nflx_pkts = nflx_pkts.sort_values(by="timestamp")
nflx_pkts["inter_arrival_time"] = nflx_pkts["timestamp"].diff().fillna(0)
print(nflx_pkts["inter_arrival_time"].describe())

#burstiness: standard deviation of inter-arrival times
burstiness = nflx_pkts["inter_arrival_time"].std()
print(f"Burstiness (std of inter-arrival times): {burstiness}")

#protocol distribution which daescribes 
protocol_counts = nflx_pkts["protocol"].value_counts(normalize=True)
print("Protocol Distribution:")
print(protocol_counts)

#flow durations and packet counts per flow
nflx_pkts["flow_id"] = nflx_pkts.apply(lambda row: f"{row['src_ip']}-{row['dst_ip']}-{row['src_port']}-{row['dst_port']}-{row['protocol']}", axis=1)
flow_stats = nflx_pkts.groupby("flow_id").agg(
    flow_duration=pd.NamedAgg(column="timestamp", aggfunc=lambda x: x.max() - x.min()),
    packet_count=pd.NamedAgg(column="timestamp", aggfunc="count")
).reset_index()
print("Flow Statistics:")
print(flow_stats.describe())


**Write a brief justification for the features that you have chosen.**

### Inferring Segment downloads

In addition to the features that you could generate using the `netml` library or similar, add to your feature vector a "segment downloads rate" feature, which indicates the number of video segments downloaded for a given time window.

Note: If you are using the `netml` library, generating features with `SAMP` style options may be useful, as this option gives you time windows, and you can then simply add the segment download rate to that existing dataframe.

In [ ]:
#add to your feature vector a "segment downloads rate" feature, which indicates the number of video segments downloaded for a given time window.
nflx_pkts["time_window"] = (nflx_pkts["timestamp"] // 10) * 10  # 10-second windows
segment_download_rate = nflx_pkts.groupby("time_window").size().reset_index(name="segment_download_rate")
nflx_pkts = nflx_pkts.merge(segment_download_rate, on="time_window", how="left")
print(nflx_pkts[["timestamp", "time_window", "segment_download_rate"]])

## Part 2: Video Quality Inference

You will now load the complete video dataset from a previous study to train and test models based on these features to automatically infer the quality of a streaming video flow.

For this part of the assignment, you will need two pickle files, which we provide for you by running the code below:

```

!gdown 'https://drive.google.com/uc?id=1N-Cf4dJ3fpak_AWgO05Fopq_XPYLVqdS' -O netflix_session.pkl
!gdown 'https://drive.google.com/uc?id=1PHvEID7My6VZXZveCpQYy3lMo9RvMNTI' -O video_dataset.pkl

```

### Load the File

Load the video dataset pickle file.

In [ ]:
!gdown 'https://drive.google.com/uc?id=1N-Cf4dJ3fpak_AWgO05Fopq_XPYLVqdS' -O netflix_session.pkl
!gdown 'https://drive.google.com/uc?id=1PHvEID7My6VZXZveCpQYy3lMo9RvMNTI' -O video_dataset.pkl

### Clean the File

1. The dataset contains video resolutions that are not valid. Remove entries in the dataset that do not contain a valid video resolution. Valid resolutions are 240, 360, 480, 720, 1080.

In [ ]:
import pickle
import pandas as pd

with open('../../myData/netflix_dataset.pkl', 'rb') as f:
    session_data = pickle.load(f)

print(session_data.columns.tolist())
print(session_data.head())


2. The file also contains columns that are unnecessary (in fact, unhelpful!) for performing predictions. Identify those columns, and remove them.

In [ ]:
import pandas as pd
import numpy as np

# Load and clean resolution
file_path = "../../myData/netflix_dataset.pkl"
df = pd.read_pickle(file_path)
valid_resolutions = [240, 360, 480, 720, 1080]
df_clean = df[df["resolution"].isin(valid_resolutions)].copy()

print(f"Original columns: {df_clean.shape[1]}")

# Identify columns to remove
columns_to_remove = []

# 2. Remove zero-variance columns (only for non-object types)
zero_var_cols = []
for col in df_clean.columns:
    try:
        # Only check numeric columns
        if df_clean[col].dtype in ['int64', 'float64', 'bool']:
            if df_clean[col].nunique() <= 1:
                zero_var_cols.append(col)
    except:
        pass
columns_to_remove.extend(zero_var_cols)
print(f"Zero variance columns: {zero_var_cols}")

# 3. Remove redundant 'R' columns
r_cols = [col for col in df_clean.columns if col.endswith('R')]
columns_to_remove.extend(r_cols)
print(f"Redundant 'R' columns: {len(r_cols)}")

# 4. Identify nested data structures (for reporting)..not gonna help since we want ints
nested_cols = []
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        # Check first non-null value
        sample = df_clean[col].dropna().iloc[0] if len(df_clean[col].dropna()) > 0 else None
        if isinstance(sample, (list, np.ndarray)):
            nested_cols.append(col)
columns_to_remove.extend(nested_cols)
print(f"Nested columns: {nested_cols}")

# Remove duplicates and drop
columns_to_remove = list(set(columns_to_remove))
df_final = df_clean.drop(columns=columns_to_remove)

print(f"\nTotal removed: {len(columns_to_remove)} columns")
print(f"Final dataset shape: {df_final.shape}")
print(f"\nSample of removed columns: {sorted(columns_to_remove)[:10]}")

**Briefly explain why you removed those columns.**

### Prepare Your Data

Prepare your data matrix, determine your features and labels, and perform a train-test split on your data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# ===== PREPARE FEATURES AND LABELS =====
y = df_final['resolution']  # Labels (target) - CHANGED TO RESOLUTION
X = df_final.drop(columns=['resolution'])  # Features - DROP RESOLUTION

# Remove non-numeric columns
X = X.select_dtypes(include=[np.number])

print(f"\nFeatures (X) shape: {X.shape}")
print(f"Labels (y) shape: {y.shape}")
print(f"\nTarget (resolution) distribution:")
print(y.value_counts().sort_index())

# ===== TRAIN-TEST SPLIT =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42,
    stratify=y  # ADDED: Keep same resolution distribution in train/test
)

print(f"\nTrain-Test Split Complete:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")

print(f"\nResolution distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nResolution distribution in test set:")
print(y_test.value_counts().sort_index())

### Train and Tune Your Model

1. Select a model of your choice.
2. Train the model using your training data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ===== REMOVE NON-NUMERIC COLUMNS FIRST =====
print("Checking for non-numeric columns...")
print(f"X_train shape before filtering: {X_train.shape}")

# Keep only numeric columns
X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

print(f"X_train shape after filtering: {X_train.shape}")
print(f"X_test shape after filtering: {X_test.shape}")

# ===== CHECK FOR MISSING VALUES =====
print("\nChecking data quality...")
print(f"Missing values in X_train: {X_train.isnull().sum().sum()}")
print(f"Missing values in X_test: {X_test.isnull().sum().sum()}")

# Show columns with missing values
missing_cols = X_train.columns[X_train.isnull().any()].tolist()
if missing_cols:
    print(f"\nColumns with missing values ({len(missing_cols)}):")
    for col in missing_cols[:10]:  # Show first 10
        print(f"  - {col}: {X_train[col].isnull().sum()} missing")

# ===== HANDLE MISSING VALUES =====
imputer = SimpleImputer(strategy='median')

print("\nImputing missing values with median...")
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_imputed = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"After imputation:")
print(f"  X_train missing values: {X_train_imputed.isnull().sum().sum()}")
print(f"  X_test missing values: {X_test_imputed.isnull().sum().sum()}")

# ===== TRAIN LINEAR REGRESSION MODEL =====
print("\n" + "=" * 60)
print("TRAINING LINEAR REGRESSION MODEL (80/20 SPLIT)")
print("=" * 60)

model = LinearRegression()

print(f"\nTraining on {X_train_imputed.shape[0]} samples ({X_train_imputed.shape[0]/(X_train_imputed.shape[0]+X_test_imputed.shape[0])*100:.1f}%)")
print(f"Testing on {X_test_imputed.shape[0]} samples ({X_test_imputed.shape[0]/(X_train_imputed.shape[0]+X_test_imputed.shape[0])*100:.1f}%)")
print(f"Features: {X_train_imputed.shape[1]}")

model.fit(X_train_imputed, y_train)

print("\n✓ Model training complete!")

# ===== EVALUATE ON TRAINING DATA =====
y_train_pred = model.predict(X_train_imputed)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f"\n=== Training Set Performance ===")
print(f"R² Score: {train_r2:.4f}")
print(f"RMSE: {train_rmse:.4f}p")
print(f"MAE: {train_mae:.4f}p")

# ===== EVALUATE ON TEST DATA =====
y_test_pred = model.predict(X_test_imputed)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n=== Test Set Performance ===")
print(f"R² Score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}p")
print(f"MAE: {test_mae:.4f}p")

# Show prediction examples
print(f"\n=== Sample Predictions ===")
sample_results = pd.DataFrame({
    'Actual Resolution': y_test.head(10).values,
    'Predicted Resolution': y_test_pred[:10].round(0)
})
print(sample_results.to_string(index=False))

# ===== FEATURE COEFFICIENTS (Top 10 by absolute value) =====
coefficients = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'coefficient': model.coef_
})
coefficients['abs_coefficient'] = coefficients['coefficient'].abs()
coefficients = coefficients.sort_values('abs_coefficient', ascending=False)

print(f"\n=== Top 10 Most Influential Features ===")
print(coefficients[['feature', 'coefficient']].head(10).to_string(index=False))
print(f"\nIntercept: {model.intercept_:.4f}")

# ===== CHECK FOR OVERFITTING =====
print(f"\n=== Overfitting Check ===")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Difference: {abs(train_r2 - test_r2):.4f}")
if abs(train_r2 - test_r2) < 0.05:
    print("✓ Model generalizes well (low overfitting)")
elif abs(train_r2 - test_r2) < 0.1:
    print("⚠ Slight overfitting detected")
else:
    print("❌ Significant overfitting detected")

### Tune Your Model

Perform hyperparameter tuning to find optimal parameters for your model.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

print("=" * 60)
print("HYPERPARAMETER TUNING - RIDGE REGRESSION")
print("=" * 60)

# Simple Ridge regression with different alpha values
ridge = Ridge()

param_grid = {
    'alpha': [0.1, 1, 10, 100]
}

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(
    ridge,
    param_grid,
    cv=5,
    scoring='r2',
    verbose=1
)

print("\nSearching for best alpha parameter...")
grid_search.fit(X_train_imputed, y_train)

print(f"\nBest alpha: {grid_search.best_params_['alpha']}")
print(f"Best CV R² score: {grid_search.best_score_:.4f}")

# Evaluate on test set
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_imputed)

tuned_r2 = r2_score(y_test, y_pred_tuned)
tuned_rmse = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
tuned_mae = mean_absolute_error(y_test, y_pred_tuned)

print(f"\n=== Tuned Model Performance ===")
print(f"R² Score: {tuned_r2:.4f}")
print(f"RMSE: {tuned_rmse:.4f}p")
print(f"MAE: {tuned_mae:.4f}p")

print(f"\n=== Comparison ===")
print(f"Baseline R²: {test_r2:.4f}")
print(f"Tuned R²:    {tuned_r2:.4f}")
print(f"Improvement: {tuned_r2 - test_r2:.4f}")

print(f"\n✓ Best model ready for netflix.pcap predictions!")

### Evaluate Your Model

Evaluate your model accuracy according to the following metrics:

1. Accuracy
2. F1 Score
3. Confusion Matrix
4. ROC/AUC

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("=" * 60)
print("MODEL EVALUATION - CLASSIFICATION METRICS")
print("=" * 60)

# ===== CONVERT CONTINUOUS PREDICTIONS TO CLASSES =====
print("\nConverting continuous predictions to resolution classes...")

# Define valid resolutions
valid_resolutions = [240, 360, 480, 720, 1080]

def round_to_nearest_resolution(pred):
    """Round prediction to nearest valid resolution"""
    return min(valid_resolutions, key=lambda x: abs(x - pred))

# Get predictions from best model
y_pred_continuous = best_model.predict(X_test_imputed)

# Round to nearest valid resolution
y_pred_class = np.array([round_to_nearest_resolution(p) for p in y_pred_continuous])
y_test_class = y_test.values  # Actual resolutions are already valid

print(f"Sample predictions:")
sample_df = pd.DataFrame({
    'Actual': y_test_class[:10],
    'Predicted (continuous)': y_pred_continuous[:10].round(1),
    'Predicted (rounded)': y_pred_class[:10]
})
print(sample_df.to_string(index=False))

# ===== 1. ACCURACY =====
print("\n" + "=" * 60)
print("1. ACCURACY")
print("=" * 60)

accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"\nAccuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Correct predictions: {np.sum(y_test_class == y_pred_class)} out of {len(y_test_class)}")

# ===== 2. F1 SCORE =====
print("\n" + "=" * 60)
print("2. F1 SCORE")
print("=" * 60)

f1_weighted = f1_score(y_test_class, y_pred_class, average='weighted')
f1_macro = f1_score(y_test_class, y_pred_class, average='macro')

print(f"\nWeighted F1 Score: {f1_weighted:.4f}")
print(f"Macro F1 Score: {f1_macro:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_class, y_pred_class, 
                           target_names=['240p', '360p', '480p', '720p', '1080p'],
                           zero_division=0))

# ===== 3. CONFUSION MATRIX =====
print("\n" + "=" * 60)
print("3. CONFUSION MATRIX")
print("=" * 60)

cm = confusion_matrix(y_test_class, y_pred_class, labels=valid_resolutions)
print("\n", cm)

# Visualize confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['240p', '360p', '480p', '720p', '1080p'],
           yticklabels=['240p', '360p', '480p', '720p', '1080p'],
           cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Resolution Prediction', fontsize=14, fontweight='bold')
plt.ylabel('True Resolution', fontsize=12)
plt.xlabel('Predicted Resolution', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate per-class accuracy
print("\nPer-Resolution Accuracy:")
for res in valid_resolutions:
    mask = y_test_class == res
    if np.sum(mask) > 0:
        correct = np.sum((y_test_class == res) & (y_pred_class == res))
        total = np.sum(mask)
        acc = correct / total
        print(f"  {res}p: {acc:.4f} ({correct}/{total})")

# ===== 4. ROC/AUC =====
print("\n" + "=" * 60)
print("4. ROC/AUC")
print("=" * 60)

# For regression-based predictions, we can't get probability scores directly
# But we can calculate a pseudo-probability based on distance to each class

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Convert to binary format for ROC
y_test_bin = label_binarize(y_test_class, classes=valid_resolutions)
n_classes = len(valid_resolutions)

# Calculate pseudo-probabilities based on distance
def calculate_pseudo_probabilities(predictions, valid_resolutions):
    """Convert continuous predictions to pseudo-probabilities"""
    probs = np.zeros((len(predictions), len(valid_resolutions)))
    
    for i, pred in enumerate(predictions):
        # Calculate distances to each resolution
        distances = np.abs(np.array(valid_resolutions) - pred)
        # Convert to probabilities (closer = higher probability)
        # Use softmax-like transformation
        probs[i] = np.exp(-distances / 100)  # Scale factor
        probs[i] = probs[i] / np.sum(probs[i])  # Normalize
    
    return probs

y_pred_proba = calculate_pseudo_probabilities(y_pred_continuous, valid_resolutions)

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calculate macro-average
auc_macro = np.mean(list(roc_auc.values()))

print(f"\nAUC Scores:")
for i, res in enumerate(valid_resolutions):
    print(f"  {res}p: {roc_auc[i]:.4f}")
print(f"\nMacro-average AUC: {auc_macro:.4f}")

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'green', 'orange', 'red', 'purple']

for i, (res, color) in enumerate(zip(valid_resolutions, colors)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
            label=f'{res}p (AUC = {roc_auc[i]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Resolution Prediction', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ===== SUMMARY =====
print("\n" + "=" * 60)
print("EVALUATION SUMMARY")
print("=" * 60)
print(f"Accuracy:           {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"F1 Score (weighted): {f1_weighted:.4f}")
print(f"AUC (macro):        {auc_macro:.4f}")
print(f"R² Score:           {tuned_r2:.4f}")
print(f"RMSE:               {tuned_rmse:.4f}p")

## Part 3: Predict the Ongoing Resolution of a Real Netflix Session

Now that you have your model, it's time to put it in practice!

Use a preprocessed Netflix video session to infer **and plot** the resolution at 10-second time intervals.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("=" * 60)
print("PREDICTING RESOLUTION FROM NETFLIX SESSION")
print("=" * 60)

df_full = pd.read_pickle("../../myData/netflix_dataset.pkl")

# Keep only valid resolution values
valid_resolutions = [240, 360, 480, 720, 1080]
df_sessions = df_full[df_full["resolution"].isin(valid_resolutions)].copy()

# Randomly select 50 intervals to simulate a streaming session
np.random.seed(42)
session_sample = df_sessions.sample(50).reset_index(drop=True)

print(f"Selected 50 intervals from the dataset")

X_session = session_sample.drop(columns=["resolution"])
X_session = X_session.select_dtypes(include=[np.number])
X_session = X_session.reindex(columns=X_train_imputed.columns, fill_value=0)

X_session_imputed = pd.DataFrame(
    imputer.transform(X_session),
    columns=X_session.columns
)

predictions = best_model.predict(X_session_imputed)

# Round each prediction to the nearest valid resolution
predictions = np.array([
    min(valid_resolutions, key=lambda x: abs(x - p)) for p in predictions
])


true_resolutions = session_sample["resolution"].values
time_points = np.arange(50) * 10  # every 10 seconds

plt.figure(figsize=(14, 7))

# Predicted line (blue)
plt.plot(
    time_points, predictions,
    marker="o", linewidth=2, markersize=8, color="#1f77b4",
    label="Predicted Resolution"
)

# Actual line (orange dashed)
plt.plot(
    time_points, true_resolutions,
    marker="s", linestyle="--", linewidth=2, markersize=8, color="#ff7f0e",
    label="Actual Resolution"
)

# Optional: highlight mismatched intervals in red
mismatch = predictions != true_resolutions
plt.scatter(
    time_points[mismatch], true_resolutions[mismatch],
    color="red", s=100, zorder=5, label="Mismatch Points"
)

plt.xlabel("Time (seconds)", fontsize=14, fontweight="bold")
plt.ylabel("Resolution (p)", fontsize=14, fontweight="bold")
plt.title("Netflix Streaming Resolution Over Time (Predicted vs Actual)",
          fontsize=16, fontweight="bold")
plt.grid(True, alpha=0.3)
plt.yticks(valid_resolutions, [f"{r}p" for r in valid_resolutions])
plt.xlim(left=0)
plt.legend(fontsize=12, loc="best")
plt.tight_layout()
plt.show()

print("\nPrediction summary:")
unique, counts = np.unique(predictions, return_counts=True)
for res, count in zip(unique, counts):
    print(f"  {res}p: {count} intervals ({count/50*100:.1f}%)")

accuracy = np.mean(predictions == true_resolutions)
print(f"\nExact match accuracy: {accuracy*100:.1f}%")
